In [1]:
import json
import minsearch

from openai import OpenAI

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv('../.env')

True

## Retrieval (based on minsearch)

In [4]:
with open('data/documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course'],
)
index.fit(documents)

In [7]:
q = "the course has already started, can I still enroll?"

In [8]:
boost = {
    'question': 3.0,
    'section' : 0.5
}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

## Generation

In [9]:
client = OpenAI()

In [10]:
response = client.chat.completions.create(
    model = 'gpt-4o',
    messages=[{'role': 'user', 'content': q}]
)

In [11]:
response.choices[0].message.content

"Whether you can still enroll in a course after it has started depends on the institution or platform offering the course. Here are some general steps you can take to find out:\n\n1. **Check the Course Website**: Visit the course's official website or the platform offering the course to see if they provide information on late enrollment.\n\n2. **Contact the Instructor or Administration**: Reach out directly to the course instructor or the administrative department responsible for enrollments. They can provide specific information about enrollment deadlines and possible exceptions.\n\n3. **Look for Enrollment Policies**: Some institutions have specific policies for late enrollment that might include a grace period.\n\n4. **Consider Auditing the Course**: If regular enrollment is closed, check if the course offers an option to audit. This might allow you to participate in the course without receiving formal credit.\n\n5. **Ask About Future Sessions**: If late enrollment isn't possible, i

## RAG prototype

In [12]:
def search(query):
    boost = {
    'question': 3.0,
    'section' : 0.5
    }
    
    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=10
    )

    return results

In [13]:
def build_prompt(query, search_results):
    prompt_template = """
        You are a course teaching assistant. Answer the QUESTION based only on the CONTEXT.
        If the CONTEXT does not contain the answer output NONE.
    
        QUESTION: {query}
    
        CONTEXT: {context}
    """.strip()
    
    context = ""
    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(query=query, context=context).strip()

    return prompt

In [14]:
def llm(prompt):
    response = client.chat.completions.create(
        model = 'gpt-4o',
        messages=[{'role': 'user', 'content': prompt}]
    )
    
    return response.choices[0].message.content

In [15]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [16]:
query = 'how do I run kafka?'
rag('query')

'NONE'

## Elasticsearch

In [25]:
import elasticsearch

from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

In [18]:
elasticsearch.__version__

(8, 18, 1)

In [20]:
es_client = Elasticsearch('http://localhost:9200')

In [22]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [26]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:04<00:00, 196.11it/s]


In [ ]:
query = 'I just discovered the course. Can I still join it?'

In [32]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    results_docs = []
    response = es_client.search(index=index_name, body=search_query)
    
    for hit in response['hits']['hits']:
        results_docs.append(hit['_source'])
    
    return results_docs

In [33]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [34]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [35]:
rag(query)

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."